# AC3 for Kung fu

# Part 0- Installing the Required package and importing libraries

Installing gymnasium

In [ ]:
!pip install gymnasium
!pip install "gymnasium[atari, accept-rom-license]"
!apt-get install -y swig
!pip install gymnasium[box2d]
!pip install pygame
!pip install ale-py
!pip install gym pygame box2d pyvirtualdisplay
!pip install tensorflow torch stable-baselines3
!python -m atari_py.installer

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  swig4.0
Suggested packages:
  swig-doc swig-examples swig4.0-examples swig4.0-doc
The following NEW packages will be installed:
  swig swig4.0
0 upgraded, 2 newly installed, 0 to remove and 30 not upgraded.
Need to get 1,116 kB of archives.
After this operation, 5,542 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig4.0 amd64 4.0.2-1ubuntu1 [1,110 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig all 4.0.2-1ubuntu1 [5,632 B]
Fetched 1,116 kB in 1s (1,422 kB/s)
Selecting previously unselected package swig4.0.
(Reading database ... 126210 files and directories currently installed.)
Preparing to unpack .../swig4.0_4.0.2-1ubuntu1_amd64.deb ...
Unpacking swig4.0 (4.0.2-1ubuntu1) ...
Selecting previously unselected package swig.
Preparing to unpack .../swig_4.0.2-1ubu

Importing the Libraries

In [ ]:
import cv2
import math
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.multiprocessing as mp
import torch.distributions as distributions
from torch.distributions import Categorical
import gymnasium as gym
from gymnasium import ObservationWrapper
from gymnasium.spaces import Box

# Part 1-Building the AI

Creating the architecture of Nerual Network

In [ ]:
class Network(nn.Module):
  def __init__(self, action_size):
    super(Network, self).__init__()
    self.conv1 = torch.nn.Conv2d(in_channels = 4, out_channels = 32, kernel_size = (3,3), stride = 2)
    self.conv2 = torch.nn.Conv2d(in_channels = 32, out_channels = 32, kernel_size = (3,3), stride = 2)
    self.conv3 = torch.nn.Conv2d(in_channels = 32, out_channels = 32, kernel_size = (3,3), stride = 2)
    self.flatten = torch.nn.Flatten()
    self.fc1 = torch.nn.Linear(512,128)
    self.fc2a = torch.nn.Linear(128,action_size)
    self.fc2s = torch.nn.Linear(128,1)
  def forward(self, state):
    x = self.conv1(state)
    x = F.relu(x)
    x = self.conv2(x)
    x = F.relu(x)
    x = self.conv3(x)
    x = F.relu(x)
    x = self.flatten(x)
    x = self.fc1(x)
    x = F.relu(x)
    actions_value = self.fc2a(x)
    state_value = self.fc2s(x)[0]
    return actions_value, state_value

# Part 2- Training the AI

Setting up the Enviornment

In [ ]:
class PreprocessAtari(ObservationWrapper):

  def __init__(self, env, height = 42, width = 42, crop = lambda img: img, dim_order = 'pytorch', color = False, n_frames = 4):
    super(PreprocessAtari, self).__init__(env)
    self.img_size = (height, width)
    self.crop = crop
    self.dim_order = dim_order
    self.color = color
    self.frame_stack = n_frames
    n_channels = 3 * n_frames if color else n_frames
    obs_shape = {'tensorflow': (height, width, n_channels), 'pytorch': (n_channels, height, width)}[dim_order]
    self.observation_space = Box(0.0, 1.0, obs_shape)
    self.frames = np.zeros(obs_shape, dtype = np.float32)

  def reset(self):
    self.frames = np.zeros_like(self.frames)
    obs, info = self.env.reset()
    self.update_buffer(obs)
    return self.frames, info

  def observation(self, img):
    img = self.crop(img)
    img = cv2.resize(img, self.img_size)
    if not self.color:
      if len(img.shape) == 3 and img.shape[2] == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = img.astype('float32') / 255.
    if self.color:
      self.frames = np.roll(self.frames, shift = -3, axis = 0)
    else:
      self.frames = np.roll(self.frames, shift = -1, axis = 0)
    if self.color:
      self.frames[-3:] = img
    else:
      self.frames[-1] = img
    return self.frames

  def update_buffer(self, obs):
    self.frames = self.observation(obs)

def make_env():
  env = gym.make("KungFuMasterDeterministic-v0", render_mode = 'rgb_array')
  env = PreprocessAtari(env, height = 42, width = 42, crop = lambda img: img, dim_order = 'pytorch', color = False, n_frames = 4)
  return env

env = make_env()

state_shape = env.observation_space.shape
number_actions = env.action_space.n
print("State shape:", state_shape)
print("Number actions:", number_actions)
print("Action names:", env.unwrapped.get_action_meanings())
import ale_py
print(ale_py.__version__)
import gymnasium as gym
print(gym.envs.registry.keys())

env = gym.make("KungFuMasterDeterministic-v0")
env.reset()
print("MsPacman environment loaded successfully!")

State shape: (4, 42, 42)
Number actions: 14
Action names: ['NOOP', 'UP', 'RIGHT', 'LEFT', 'DOWN', 'DOWNRIGHT', 'DOWNLEFT', 'RIGHTFIRE', 'LEFTFIRE', 'DOWNFIRE', 'UPRIGHTFIRE', 'UPLEFTFIRE', 'DOWNRIGHTFIRE', 'DOWNLEFTFIRE']
0.10.2
dict_keys(['CartPole-v0', 'CartPole-v1', 'MountainCar-v0', 'MountainCarContinuous-v0', 'Pendulum-v1', 'Acrobot-v1', 'phys2d/CartPole-v0', 'phys2d/CartPole-v1', 'phys2d/Pendulum-v0', 'LunarLander-v3', 'LunarLanderContinuous-v3', 'BipedalWalker-v3', 'BipedalWalkerHardcore-v3', 'CarRacing-v3', 'Blackjack-v1', 'FrozenLake-v1', 'FrozenLake8x8-v1', 'CliffWalking-v0', 'Taxi-v3', 'tabular/Blackjack-v0', 'tabular/CliffWalking-v0', 'Reacher-v2', 'Reacher-v4', 'Reacher-v5', 'Pusher-v2', 'Pusher-v4', 'Pusher-v5', 'InvertedPendulum-v2', 'InvertedPendulum-v4', 'InvertedPendulum-v5', 'InvertedDoublePendulum-v2', 'InvertedDoublePendulum-v4', 'InvertedDoublePendulum-v5', 'HalfCheetah-v2', 'HalfCheetah-v3', 'HalfCheetah-v4', 'HalfCheetah-v5', 'Hopper-v2', 'Hopper-v3', 'Hopper-v4

Initializing the hyperparameter

In [ ]:
learning_rate = 1e-4
discount_factor = 0.99
number_enviornments = 10

Implementing the A3C class

In [ ]:
class Agent():

   def __init__(self, action_size):
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    self.action_size = action_size
    self.network = Network(action_size).to(self.device)
    self.optimizer = torch.optim.Adam(self.network.parameters(), lr = learning_rate)

   def act(self, state):
    if state.ndim == 3:
      state = [state]
    state = torch.tensor(state, dtype = torch.float32, device = self.device)
    action_values,_ = self.network(state)
    policy = F.softmax(action_values, dim = -1)
    return np.array([np.random.choice(len(p), p=p) for p in policy.detach().cpu().numpy()])
   def step(self, state, action, reward, next_state, done):
    batch_size = state.shape[0]
    state = torch.tensor(state, dtype = torch.float32, device = self.device)
    action = torch.tensor(action, dtype = torch.float32, device = self.device)
    reward = torch.tensor(reward, dtype = torch.float32, device = self.device)
    next_state = torch.tensor(next_state, dtype = torch.float32, device = self.device)
    done = torch.tensor(done, dtype = torch.bool, device = self.device).to(dtype = torch.float32)
    action_values, state_values = self.network(state)
    _, next_state_values = self.network(next_state)
    target_state_value = reward + discount_factor * next_state_values * (1-done)
    advantage = target_state_value - state_values
    probs = F.softmax(action_values, dim = -1)
    log_probs = F.log_softmax(action_values, dim = -1)
    entropy = -torch.sum(probs * log_probs, dim = -1)
    batch_idx = np.arange(batch_size)
    logp_actions = log_probs[batch_idx, action.long()]
    actor_loss = -(logp_actions * advantage.detach() + 0.01 * entropy).mean()
    critic_loss = F.mse_loss(state_values, target_state_value.detach())
    total_loss = actor_loss + critic_loss
    self.optimizer.zero_grad()
    total_loss.backward()
    self.optimizer.step()


Initializing the A3C agent

In [ ]:
agent = Agent(number_actions)

Evaluating our A3C agent on a certain number of eposide

In [ ]:
def evaluate(agent, env, n_episodes = 1):
  episodes_rewards = []
  for _ in range(n_episodes):
    env_eval = make_env()
    state, _ = env_eval.reset()
    total_reward = 0
    while True:
      action = agent.act(state)
      state, reward, done, info, _ = env_eval.step(action[0])
      total_reward += reward
      if done:
        break
    episodes_rewards.append(total_reward)
  return episodes_rewards


Managing multiple enviornment simultaneously

In [ ]:
class EnvBatch:
  def __init__ (self, n_envs = 10):
    self.envs = [make_env() for _ in range(n_envs)]

  def reset(self):
    _states = []
    for env in self.envs:
      _states.append(env.reset()[0])
    return np.array(_states)
  def step(self, actions):
    next_states, rewards, dones, infos, _= map(np.array, zip(*[env.step(a) for env, a in zip(self.envs, actions)]))
    for i in range(len(self.envs)):
      if dones[i]:
        next_states[i] = self.envs[i].reset()[0]
    return next_states, rewards, dones, infos

Training the A3C agent

In [ ]:
import tqdm # import for showing progress_bar

# Initialize the environment batch
env_batch = EnvBatch(number_enviornments) # This creates multiple parallel environments
batch_states = env_batch.reset() # Resets all environments and returns their initial states.

with tqdm.trange(0,3001) as progress_bar: # This runs for 3,001 iterations with a progress bar for tracking
  for i in progress_bar:
    batch_actions = agent.act(batch_states) # The agent selects actions based on the current state
    batch_next_states, batch_rewards, batch_dones, _ = env_batch.step(batch_actions) # The environments execute those actions and return
    batch_rewards *= 0.01  #The reward is scaled down by 0.01. This helps stabilize training in environments with high rewards.
    agent.step(batch_states, batch_actions, batch_rewards, batch_next_states, batch_dones)  # The A3C agent updates its policy and value function using this experience
    batch_states = batch_next_states # Moves to the next set of states
    if i % 1000 == 0: # Every 1,000 iterations, it evaluates the agent by running 10 test episodes
      print("Average agent reward:", np.mean(evaluate(agent, env, n_episodes = 10)))  # Prints the average reward of the agent.


/usr/local/lib/python3.11/dist-packages/gymnasium/envs/registration.py:519: DeprecationWarning: WARN: The environment KungFuMasterDeterministic-v0 is out of date. You should consider upgrading to version `v4`.
  logger.deprecation(
  0%|          | 0/3001 [00:00<?, ?it/s]<ipython-input-8-060d897ab165>:33: UserWarning: Using a target size (torch.Size([10])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  critic_loss = F.mse_loss(state_values, target_state_value.detach())
  0%|          | 5/3001 [00:42<5:14:01,  6.29s/it] 

Average agent reward: 650.0


 33%|███▎      | 1005/3001 [01:45<1:07:45,  2.04s/it]

Average agent reward: 510.0


 67%|██████▋   | 2005/3001 [02:53<33:51,  2.04s/it]

Average agent reward: 750.0


100%|██████████| 3001/3001 [03:56<00:00, 12.68it/s]

Average agent reward: 740.0


Visualizing the results

In [ ]:
import glob
import io
import base64
import imageio
from IPython.display import HTML, display
from moviepy.editor import ImageSequenceClip

def show_video_of_model(agent, env):

  env = gym.make("KungFuMasterDeterministic-v0", render_mode = 'rgb_array')  #Added render mode
  env = PreprocessAtari(env, height = 42, width = 42, crop = lambda img: img, dim_order = 'pytorch', color = False, n_frames = 4)
  state, _ = env.reset()
  done = False
  frames = []
  while not done:
    frame = env.render()
    frames.append(frame)
    action = agent.act(state)
    state, reward, done, _, _ = env.step(action[0])
  env.close()
  clip = ImageSequenceClip(frames, fps=30)
  clip.write_videofile('video.mp4')

show_video_of_model(agent, env)

def show_video():
    mp4list = glob.glob('*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")

show_video()

  logger.deprecation(



Moviepy - Building video video.mp4.
Moviepy - Writing video video.mp4



Moviepy - Done !
Moviepy - video ready video.mp4
